In [10]:
#%cd /content/
#!rm -rf fever-scorer

In [11]:
!git clone -b release-v2.0 https://github.com/sheffieldnlp/fever-scorer.git

Cloning into 'fever-scorer'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 224 (delta 0), reused 0 (delta 0), pack-reused 219 (from 1)
Receiving objects: 100% (224/224), 1.13 MiB | 1.93 MiB/s, done.
Resolving deltas: 100% (110/110), done.


In [12]:
%cd fever-scorer

/content/fever-scorer/fever-scorer


In [13]:
!pip install -r requirements.txt


In [14]:
# Open /setup.py and add 'license="MIT"' on line 12, then overwrite the file
import os
with open('setup.py', 'r') as f:
    lines = f.readlines()
    lines[11] = 'license="MIT"\n'
with open('setup.py', 'w') as f:
    f.writelines(lines)
    f.close()
    print("setup.py updated")


setup.py updated


In [15]:
!pip install .

Processing /content/fever-scorer/fever-scorer
  Preparing metadata (setup.py) ... done
  Created wheel for fever-scorer: filename=fever_scorer-0.0.0-py3-none-any.whl size=8288 sha256=4ba6e7b6e0742e5445329a53c7e4e225784dd9c0e2dfc92cdfbfa3a81ce6491b
  Stored in directory: /root/.cache/pip/wheels/b6/96/15/0962bf09ded2b51cf178b2b5261b279ba126ecc78d08c849de
Successfully built fever-scorer
  Attempting uninstall: fever-scorer
    Found existing installation: fever-scorer 0.0.0
    Uninstalling fever-scorer-0.0.0:
      Successfully uninstalled fever-scorer-0.0.0


In [16]:
!pip install rouge-score

In [17]:
import pandas as pd
import nltk  # Make sure NLTK is installed and data downloaded (e.g., nltk.download('punkt'))
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
import openai  # For LLM interaction
from openai import OpenAI
import numpy as np
from nltk import Tree, pos_tag, word_tokenize, ne_chunk
from nltk.corpus import stopwords
import numpy as np
from fever.scorer import fever_score # Import the FEVER scorer
from nltk import RegexpParser
import json

# Download the necessary NLTK data files
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker')
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')
nltk.download('stopwords')
nltk.download('treebank')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker_tab is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punk

True

In [18]:


# Sample instances
instances = [
    {
        "label": "REFUTES",
        "predicted_label": "REFUTES",
        "predicted_evidence": [["Page1", 1], ["Page3", 2]],
        "evidence": [
            [
                [None, None, "Page1", 1],
                [None, None, "Page3", 1],
                [None, None, "Page3", 2],
            ],
        ],
    },
    {
        "label": "SUPPORTS",
        "predicted_label": "SUPPORTS",
        "predicted_evidence": [["Page3", 3]],
        "evidence": [
            [
                [None, None, "Page3", 3]
            ]
        ],
    },
    {
        "label": "NOT ENOUGH INFO",
        "predicted_label": "NOT ENOUGH INFO",
        "predicted_evidence": [],
        "evidence": [],
    },
]

# Calculate scores
strict_score, label_accuracy, precision, recall, f1 = fever_score(instances)

# Display results
print(f"Strict Score: {strict_score:.2f}")
print(f"Label Accuracy: {label_accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Strict Score: 0.67
Label Accuracy: 1.00
Precision: 1.00
Recall: 0.50
F1 Score: 0.67


In [19]:
# Mount google drive
from google.colab import drive
import gc

drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
from google.colab import userdata
api_key = userdata.get('openaikey')
client = OpenAI(api_key=api_key)

In [30]:
%cd ../drive/My Drive/SUNY_Poly_DSA598/

/content/drive/My Drive/SUNY_Poly_DSA598


In [31]:
!ls -a

archive			  .git				  presentation
datasets		  .gitignore			  transcribe_voice_notes.ipynb
FEVER_set_creation.ipynb  liar_gpt4omini_base_eval.ipynb  work_documents
FEVER_set_update.ipynb	  Module_2_dev.ipynb


In [32]:
%cd ./datasets/FEVER/

/content/drive/My Drive/SUNY_Poly_DSA598/datasets/FEVER


In [33]:
!ls -a

AVeriTeC   fever2-adversarial.jsonl	    fever-train.jsonl  paper_dev.jsonl	 tabular_sets
.DS_Store  feverous_train_challenges.jsonl  GPT_sets	       paper_test.jsonl  wiki-pages


In [34]:
def load_jsonl(file_path, encoding='utf-8'):
    """Loads a JSON Lines file into a list of Python objects."""
    data = []
    with open(file_path, 'r', encoding=encoding) as f:  # Specify encoding for safety
        for line in f:
            data.append(json.loads(line))  # Parse each line individually
    return data

In [35]:
# Data paths (replace with your actual paths if different)
fever_path = "./datasets/FEVER/"
train_clf_path = f"tabular_sets/tabular_clf_paper_dev_train/v1_segmented_sentIDs_n3461_04-04_002.csv"
valid_clf_path = f"tabular_sets/tabular_clf_paper_dev_valid/v1_segmented_sentIDs_n1482_04-04_002.csv"
train_sentEx_path = f"tabular_sets/tabular_sentEx_paper_dev_train/v1_segmented_sentIDs_n3461_04-04_002.csv"
valid_sentEx_path = f"tabular_sets/tabular_sentEx_paper_dev_valid/v1_segmented_sentIDs_n1482_04-04_002.csv"
test_path = f"paper_test.jsonl"
train_path = f"paper_dev.jsonl"

# Load datasets
#train_clf = pd.read_csv(train_clf_path)
#valid_clf = pd.read_csv(valid_clf_path)
train_sentEx = pd.read_csv(train_sentEx_path)
valid_sentEx = pd.read_csv(valid_sentEx_path)
test_jsonl = load_jsonl(test_path)
train_jsonl = load_jsonl(train_path)

In [36]:
# Show the distribution of labels
print(f"Train set label distribution:")
print(train_sentEx['label'].value_counts())
print(f"Valid set label distribution:")
print(valid_sentEx['label'].value_counts())

# Balance the labels by reducing each to the minimum count
min_count = min(train_sentEx['label'].value_counts())
train_sentEx = train_sentEx.groupby('label').apply(lambda x: x.sample(min_count)).reset_index(drop=True)
min_count = min(valid_sentEx['label'].value_counts())
valid_sentEx = valid_sentEx.groupby('label').apply(lambda x: x.sample(min_count)).reset_index(drop=True)

print(f"Train set label distribution after balancing:")
print(train_sentEx['label'].value_counts())
print(f"Valid set label distribution after balancing:")
print(valid_sentEx['label'].value_counts())

Train set label distribution:
label
SUPPORTS           1156
REFUTES            1156
NOT ENOUGH INFO    1149
Name: count, dtype: int64
Valid set label distribution:
label
SUPPORTS           495
REFUTES            495
NOT ENOUGH INFO    488
Name: count, dtype: int64
Train set label distribution after balancing:
label
NOT ENOUGH INFO    1149
REFUTES            1149
SUPPORTS           1149
Name: count, dtype: int64
Valid set label distribution after balancing:
label
NOT ENOUGH INFO    488
REFUTES            488
SUPPORTS           488
Name: count, dtype: int64


<ipython-input-36-03091368d446>:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_sentEx = train_sentEx.groupby('label').apply(lambda x: x.sample(min_count)).reset_index(drop=True)
<ipython-input-36-03091368d446>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  valid_sentEx = valid_sentEx.groupby('label').apply(lambda x: x.sample(min_count)).reset_index(drop=True)


## Module 2

In [79]:
def module_2_semi_supervised_distillation(claim, documents, entities, keywords, sim_thresh=0.2, verbose=0, debug=False):
    """
    Module 2: Semi-supervised Distillation for sentence extraction.

    Args:
        claim (str): The input claim.
        documents (list of str): List of retrieved documents (full text).
        entity (str):  The main entity in the claim (or None if not found).
        keywords (list of str):  Top keywords from the claim.

    Returns:
        tuple: (list of str, str):  A list of extracted evidence sentences and the exit status ("NOT ENOUGH INFO" or "OK").
    """


    extracted_sentences = []
    max_iterations = min(5, len(documents)) # This will be determined later with actual number of sentences across all docs
    if debug:
      print(f"DEBUG 2.1.1:")
      print(f"\tNumber of documents: {len(documents)}")
      print(f"\tMax iterations: {max_iterations}")
      print("-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")

    """
    if entities == []:
      if verbose == 1:
        print("No entities found, early exiting...")
        return [], "NOT ENOUGH INFO" # Early exit if no entity
    """

    # convert entities to string
    if not entities:
      entities = ""
      # prompts
      prompts = {
            "init": f"Retrieve sentences from the list that either support or refute the following claim. Specifically, focus on sentences mentioning {keywords}. Order the sentences by relevance, highest first, and return a list separated by the return character. If there are no relevant sentences, respond with 'NOT ENOUGH INFO'. DO NOT CREATE ANY SENTENCES THAT ARE NOT IN THE PROVIDED LIST, AND DO NOT TRUNCATE THE SENTENCE.",
            "followup": f"You didn’t find enough sentences. Find additional (new) sentences that are relevant to key points in the claim. Order the sentences by relevance, highest first, and return a list separated by the return character. If there are no relevant sentences, respond with 'NOT ENOUGH INFO'. DO NOT CREATE ANY SENTENCES THAT ARE NOT IN THE PROVIDED LIST, AND DO NOT TRUNCATE THE SENTENCE.",
        }
    else:
      entities = ", ".join(entities)
      # prompts
      prompts = {
            "init": f"Retrieve sentences from the list that either support or refute the following claim. Specifically, focus on sentences mentioning {entities} or {keywords}. Order the sentences by relevance, highest first, and return a list separated by the return character. If there are no relevant sentences, respond with 'NOT ENOUGH INFO'. DO NOT CREATE ANY SENTENCES THAT ARE NOT IN THE PROVIDED LIST, AND DO NOT TRUNCATE THE SENTENCE.",
            "followup": f"You didn’t find enough sentences. Find additional (new) sentences that that are relevant to key points in the claim. Order the sentences by relevance, highest first, and return a list separated by the return character. If there are no relevant sentences, respond with 'NOT ENOUGH INFO'. DO NOT CREATE ANY SENTENCES THAT ARE NOT IN THE PROVIDED LIST, AND DO NOT TRUNCATE THE SENTENCE.",
      }
      if debug:
        print(f"DEBUG 2.1.2:")
        print(f"\tClaim: {claim}")
        print(f"\tEntities: {entities}")
        print(f"\tKeywords: {keywords}")
        print("-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")

    for iteration in range(max_iterations):
      # 2.2 Prompt Selection (no agent - programmatic)
      if iteration == 0:
          prompt = prompts["init"]
      else:
          prompt = prompts["followup"]


      # 2.3 Sentence Extraction (with pre-filtering)
      # Drop empty sentences from the list
      documents = [doc for doc in documents if doc.strip()]
      #filtered_text = sliding_window_filter(documents, entities, keywords)
      # Simply form the list of document strings into one string joined by \n for now
      filtered_text = "\n".join(documents)
      if debug:
        print(f"DEBUG 2.3.1:")
        print(f"\tFiltered text: {filtered_text}")
        print("-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")
      new_sentences = extract_sentences_with_llm(claim, filtered_text, prompt, debug)

      if debug:
        print(f"DEBUG 2.3.3:")
        print(f"\tNumber of retrieved sentences on iteration {iteration}: {len(new_sentences)}")
        print(f"\tNew sentences: {new_sentences}")
        print("-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")

      if new_sentences != ["NOT ENOUGH INFO"]:  # LLM found no relevant sentences
        if verbose == 1:
          print(f"2.3: LLM found sentences on iteration {iteration}.")

        # 2.4 Similarity Comparison and Thresholding
        new_sentences = similarity_thresholding(claim, new_sentences, sim_thresh, True)

        if verbose == 1:
          print(f"\tExtracted sentences: {new_sentences}")
          print(f"\tNumber of extracted sentences: {len(new_sentences)}")
          print(f"\tMax iterations: {max_iterations}")
          print("-------------------------------------------------------------")

        extracted_sentences.extend(new_sentences)  # Add new sentences to the list

        if debug:
          print(f"DEBUG 2.4.2:")
          print(f"\tAdded {len(new_sentences)} new sentences on iteration {iteration}.")
          print(f"\tTotal extracted sentences: {len(extracted_sentences)}")
          print(f"\tExtracted sentences: {extracted_sentences}")
          print("-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")
          print(f"Reached the end of the loop, iteration: {iteration}")
          print("#-------------------------------------------------------------#")
        if len(extracted_sentences) >= min(max_iterations, 5):
            if verbose == 1:
                print(f"2.4: Found enough sentences, exiting...")
                print("-------------------------------------------------------------")
            return extracted_sentences, "OK"

        #Check for final early exit
        elif len(extracted_sentences) < min(max_iterations, 5) and iteration == max_iterations: # Pilot study will determine best practices here
          if verbose == 1:
            print(f"2.4: REACHED MAX ITERATIONS, BUT FEWER THAN THE MAXIMUM POSSIBLE SENTENCES FOUND, EARLY EXITING WITH WHATEVER WE HAVE SO FAR")
            print("-------------------------------------------------------------")
          return extracted_sentences, "OK"
      else:
        if iteration == 0:  # No sentences found in the first iteration
          if iteration+1 != max_iterations:
            if verbose == 1:
              print(f"2.3: LLM claims to find NOT ENOUGH INFO on first iteration (focusing on entities), reprompting without entities...")
              print("-------------------------------------------------------------")
            continue
          else:
            if verbose == 1:
              print(f"2.3: LLM claims to find NOT ENOUGH INFO on first iteration and ONLY 1 ITERATION POSSIBLE, EARLY EXITING...")
              print("-------------------------------------------------------------")
            return [], "NOT ENOUGH INFO"  # Early exit if no relevant sentences in first pass.
        elif iteration == 1 and len(extracted_sentences) != 0:  # No sentences found in the second iteration, but at least one found in the first iteration
          if verbose == 1:
            print(f"2.3: LLM found no relevant sentences in the second iteration, but found at least one in the first iteration. Exiting with what we have...")
            print("-------------------------------------------------------------")
          return extracted_sentences, "OK"
        elif iteration == 1 and len(extracted_sentences) == 0:  # No sentences found in the second iteration and no sentences found in the first iteration
          if verbose == 1:
            print(f"2.3: LLM found no relevant sentences in the second iteration and no sentences found in the first iteration, early exiting...")
            print("-------------------------------------------------------------")
          return [], "NOT ENOUGH INFO"
        else:
          if verbose == 1:
            print(f"2.3: LLM claims to find NOT ENOUGH INFO after 2 iterations, early exiting...")
            print("-------------------------------------------------------------")
          return [], "NOT ENOUGH INFO"  # Early exit if no relevant sentences in third pass.


def sliding_window_filter(documents, entity, keywords, debug=False):
    """
    Performs sliding window filtering based on entity and keywords.

    Args:
        documents (list of str):  List of documents.
        entity (str): The entity to match.
        keywords (list of str): The keywords to match.

    Returns:
        str: Concatenated filtered text.
    """
    # TODO: Implement sliding window filtering logic (using NLTK, spaCy, etc.).
    # This function should concatenate sentences from windows that contain entity/keywords.
    filtered_sentences = []

    for document in documents:
        sentences = nltk.sent_tokenize(document)
        window_size = 3  # Experiment with different window sizes.
        for i in range(len(sentences) - window_size + 1):
            window = sentences[i : i + window_size]
            window_text = " ".join(window)
            if entity in window_text and any(keyword in window_text for keyword in keywords):
                filtered_sentences.extend(window) # Add matching sentences from window to filtered sentences

    filtered_text = "\n".join(filtered_sentences)
    return filtered_text

def extract_sentences_with_llm(claim, filtered_text, prompt, debug=False):
    """
    Extracts sentences using an LLM.

    Args:
        claim (str): The input claim.
        filtered_text (str): The pre-filtered text.
        prompt (str): The prompt for the LLM.

    Returns:
        list of str: Extracted sentences.
    """
    # TODO: Implement LLM interaction (using OpenAI API, etc.).
    # Use the provided prompt and filtered_text to extract sentences with the LLM.

    if debug:
      print(f"DEBUG 2.3.2:")
      print(f"\tPrompt: {prompt}")
      print(f"\tFiltered text: {filtered_text}")
      print("-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")

    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that extracts evidence sentences."},
            {"role": "user", "content": f"{prompt}\nClaim: {claim}\nText: {filtered_text}"},
        ],
        max_tokens=1024,
        n=1,
        stop=None,  # Or a suitable stop sequence
        temperature=0.9,  # Adjust as needed
    )
    extracted_sentences_raw = response.choices[0].message.content
    extracted_sentences = extracted_sentences_raw.split('\n')  # Assuming sentences are separated by newlines

    return extracted_sentences


def similarity_thresholding(claim, sentences, threshold=0.0, debug=False):
    """
    Thresholds sentences based on similarity to the claim.

    Args:
        claim (str): The input claim.
        sentences (list of str): The sentences to threshold.

    Returns:
        list of str:  The sentences that meet the similarity threshold.
    """
    # TODO: Implement similarity calculations (ROUGE, TF-IDF, cosine similarity).
    # Return only the sentences that exceed the defined threshold(s).

    filtered_sentences = []

    vectorizer = TfidfVectorizer()  # Initialize TF-IDF vectorizer
    claim_tfidf = vectorizer.fit_transform([claim])
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

    for sentence in sentences:
        sentence_tfidf = vectorizer.transform([sentence])
        cosine_sim = cosine_similarity(claim_tfidf, sentence_tfidf)[0][0]
        rouge_scores = scorer.score(claim, sentence)
        rouge1_score = rouge_scores['rouge1'].fmeasure  # Example: Use ROUGE-1 F1-score

        if debug:
          print(f"DEBUG 2.4.1:")
          print(f"\tClaim: {claim}")
          print(f"\tSentence: {sentence}")
          print(f"\tCosine Similarity: {cosine_sim}")

        if cosine_sim >= threshold and rouge1_score >= threshold:  # Example: Combine cosine similarity and ROUGE
            filtered_sentences.append(sentence)
            if debug:
              print(f"\tAdded sentence to filtered sentences: {sentence}")
              print("-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")
        else:
          if debug:
            print(f"\tSentence did not meet similarity threshold: {sentence}")
            print("-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")

    return filtered_sentences


# Entity and keywords extraction function
def extract_entities(text):
    """
    Extracts entities from the text using NLTK's Named Entity Chunker.

    Args:
        text (str): The input text.

    Returns:
        list of str: List of extracted entities.
    """
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.lower() not in stop_words]
    tagged_tokens = pos_tag(tokens)
    named_entities = ne_chunk(tagged_tokens)

    entities = []
    for subtree in named_entities:
        if isinstance(subtree, Tree):
            entity = " ".join([word for word, tag in subtree.leaves()])
            entities.append(entity)

    return entities


# Keyword extraction function
def extract_keywords(text):
    """
    Extracts keywords from the text using TF-IDF.

    Args:
        text (str): The input text.

    Returns:
        list of str: List of extracted keywords.
    """
    vectorizer = TfidfVectorizer(stop_words='english', max_features=10)  # Adjust max_features as needed
    tfidf_matrix = vectorizer.fit_transform([text])
    feature_names = vectorizer.get_feature_names_out()
    dense = tfidf_matrix.todense()
    denselist = dense.tolist()
    # Convert the list to a NumPy array to use argsort()

    dense_array = np.array(denselist[0])
    keywords = [feature_names[i] for i in dense_array.argsort()[-5:]]  # Get top 5 keywords

    return keywords

## Module 1

In [48]:
# Module 1: Document retrieval from wikipedia based on keywords, entities, and claim
import requests
from bs4 import BeautifulSoup
import re
import ast


def query_generator(claim, keywords, entities, debug=False):
    """
    Generates a query for Wikipedia based on the claim, keywords, and entities.

    Args:
        claim (str): The input claim.
        keywords (list of str): The keywords to include in the query.
        entities (list of str): The entities to include in the query.

    Returns:
        str: The generated query.
    """
    prompt = f"Generate a Wikipedia query based on the claim: '{claim}'. Include keywords: {', '.join(keywords)} and entities: {', '.join(entities)}. Respond only with the query as a string on a single line."
    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates Wikipedia queries."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=512,  # Adjust as needed
        n=1,
        stop=None,  # Or a suitable stop sequence
        temperature=0.9,  # Adjust as needed
    )
    query = response.choices[0].message.content
    return query

def retrieve_documents_from_wikipedia(claim, keywords, entities, debug=False):
    """
    Retrieves documents from Wikipedia based on the claim, keywords, and entities.

    Args:
        claim (str): The input claim.
        keywords (list of str): The keywords to include in the query.
        entities (list of str): The entities to include in the query.

    Returns:
        list of str: Retrieved documents.
    """
    query = query_generator(claim, keywords, entities, debug=False)
    if debug:
      print(f"DEBUG 1.2.1:")
      print(f"\tQuery: {query}")
      print("-_-_-_-_-_-_-_-_-_-_-_-_-_-")
    # Perform a Wikipedia search
    search_url = f"https://en.wikipedia.org/w/index.php?search={requests.utils.quote(query)}"
    response = requests.get(search_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # Extract links to Wikipedia pages
    links = soup.find_all('a', href=re.compile(r'^/wiki/'))
    documents = []
    # Get just the introduction of each page
    for link in links:
        page_url = f"https://en.wikipedia.org{link['href']}"
        response = requests.get(page_url)
        data = response.json
        if 'parse' in data:
          page_content = data['parse']['text']['*']
          # Extract the introduction section (the first paragraph element)
          intro_section = re.search(r'<p>(.*?)</p>', page_content, re.DOTALL)
          if intro_section:
            intro_text = intro_section.group(1) # Get the text inside the first <p> tag
            # Remove HTML tags
            intro_text = re.sub(r'<.*?>', '', intro_text)
            # Remove references
            intro_text = re.sub(r'\[.*?\]', '', intro_text)
            # Remove digits that are preceded by any letter, period, colon, semicolon, endash (–), and emdash(—) and followed by a space
            intro_text = re.sub(r'(?<=[a-zA-Z0-9\.\:\;\–\—])\d+(?=\s)', '', intro_text)
            # Convert encoded html entities to unicode (e.g., &amp; to &)
            intro_text = re.sub(r'&[a-zA-Z0-9#]+;', '', intro_text)
            # Remove extra whitespace
            intro_text = re.sub(r'\s+', ' ', intro_text).strip()
            documents.append(intro_text)
        else:
            print(f"Error: No parse data found for {page_url}")
    if debug:
      print(f"DEBUG 1.2.2:")
      print(f"\tNumber of documents retrieved: {len(documents)}")
      print("-_-_-_-_-_-_-_-_-_-_-_-_-")
    return documents

index = 0
def get_test_claim(df, mode, verbose=0, debug=False):
    """
    In "test" mode, gets data for a claim by matching a row in the df (which contains the claim and the wikipedia text data) to the JSONL object
    (which contains the claim and the evidence references) by the claim, returning the claim, label, evidence sentences, documents, and evidence references.

    In 'live' mode, gets data for a claim by generating a query and retrieving documents from Wikipedia. THIS MODE IS NOT COMPATIBLE WITH FEVER SCORING.

    EXAMPLE:
    {"id": 113501, "verifiable": "NOT VERIFIABLE", "label": "NOT ENOUGH INFO", "claim": "Grease had bad reviews.", "evidence": [[[133128, null, null, null]]]}
    {"id": 163803, "verifiable": "VERIFIABLE", "label": "SUPPORTS", "claim": "Ukrainian Soviet Socialist Republic was a founding participant of the UN.", "evidence": [[[296950, 288668, "Ukrainian_Soviet_Socialist_Republic", 7]], [[298602, 290067, "Ukrainian_Soviet_Socialist_Republic", 7], [298602, 290067, "United_Nations", 0]], [[300696, 291816, "Ukrainian_Soviet_Socialist_Republic", 7]], [[344347, 327887, "Ukrainian_Soviet_Socialist_Republic", 7]], [[344994, 328433, "Ukrainian_Soviet_Socialist_Republic", 7]], [[344997, 328435, "Ukrainian_Soviet_Socialist_Republic", 7]]]}
    {"id": 70041, "verifiable": "VERIFIABLE", "label": "SUPPORTS", "claim": "2 Hearts is a musical composition by Minogue.", "evidence": [[[225394, 230056, "2_Hearts_-LRB-Kylie_Minogue_song-RRB-", 0]], [[317953, 306972, "2_Hearts_-LRB-Kylie_Minogue_song-RRB-", 0]], [[319638, 308345, "2_Hearts_-LRB-Kylie_Minogue_song-RRB-", 0]], [[319643, 308348, "2_Hearts_-LRB-Kylie_Minogue_song-RRB-", 0]]]}
    {"id": 202314, "verifiable": "VERIFIABLE", "label": "REFUTES", "claim": "The New Jersey Turnpike has zero shoulders.", "evidence": [[[238335, 240393, "New_Jersey_Turnpike", 15]]]}
    """


    global index
    claim = df.iloc[index]['claim']
    documents = df.iloc[index]['full_text']
    documents = documents.split('\n')
    label = df.iloc[index]['label']
    keywords = extract_keywords(claim)  # Extract keywords from the claim
    evidence_items = df.iloc[index]['evidence_sentences']

    if debug:
      print(f"DEBUG 1.1.1:")
      print(f"Evidence items: {evidence_items}")
    # Evidence items are in the format (sentence, page_title, sentence_id, entities[entity1, entity2, ...])
    '''
    [('Despite their San Francisco Bay Area origins , they played in a Southern rock style , with lyrics about bayous , catfish , the Mississippi River , and other popular elements of Southern United States iconography , as well as political and socially-conscious lyrics about topics including the Vietnam War .', 'Creedence_Clearwater_Revival', 3, ['Vietnam War', 'Southern rock', 'San Francisco Bay Area', 'Opposition to United States involvement in the Vietnam War', 'rock', 'Mississippi River', 'rock music']), ('Creedence Clearwater Revival , often informally abbreviated to Creedence or CCR , was an American rock band active in the late 1960s and early 1970s .', 'Creedence_Clearwater_Revival', 0, ['rock', 'rock music']), ('Their musical style encompassed the roots rock , swamp rock , and blues rock genres .', 'Creedence_Clearwater_Revival', 2, ['roots rock', 'rock', 'blues rock', 'rock music', 'swamp rock'])]
    '''
    index += 1
    # Return essential information (we only need the documents, keywords, entities, claim, and label for the NOT ENOUGH INFO case, since we don't need to extract evidence sentences)
    if label == "NOT ENOUGH INFO":
        evidence_items = []
        entities = extract_entities(claim)  # Extract entities from the claim
        if verbose == 1:
          print(f"-------------------------------------------------------------")
          print(f"In get_test_claim, running in {mode} mode...")
          print(f"Claim: {claim}")
          print(f"Label: {label}")
          print(f"Evidence items: {evidence_items}")
          print(f"Documents: {documents}")
          print(f"Entities: {entities}")
          print(f"Keywords: {keywords}")
          print(f"-------------------------------------------------------------")
        return claim, label, evidence_items, documents, keywords, entities

    entities = []
    # Use ast.literal_eval to convert the string representation of the list to an actual list
    evidence_items = ast.literal_eval(evidence_items)
    # Extract entities from the evidence items
    for item in evidence_items:
        entities.extend(item[3])
        if debug:
          print(f"DEBUG 1.1.2:")
          print(f"\tEvidence item: {item}")
          print(f"\tEntities: {item[3]}")
          print("-_-_-_-_-_-_-_-_-_-_-_-_-_-")
    # Remove duplicates
    entities = list(set(entities))
    # Remove empty strings
    entities = [entity for entity in entities if entity]

    if verbose == 1:
        print(f"-------------------------------------------------------------")
        print(f"In get_test_claim, running in {mode} mode...")
        print(f"Claim: {claim}")
        print(f"Label: {label}")
        print(f"Evidence items: {evidence_items}")
        print(f"Documents: {documents}")
        print(f"Entities: {entities}")
        print(f"Keywords: {keywords}")
        print(f"-------------------------------------------------------------")


    return claim, label, evidence_items, documents, keywords, entities



In [ ]:
'''
For eventual "live" testing of the system:

    if mode == 'live':
        # Generate a query and retrieve documents from Wikipedia
        documents = retrieve_documents_from_wikipedia(claim, keywords, entities, debug=debug)
        if debug:
          print(f"DEBUG 1.2.3, Live mode:")
          print(f"\tDocuments: {documents}")
          print("-_-_-_-_-_-_-_-_-_-_-_-_-")

    elif mode == 'test':
        # Use the documents from the df
        if debug:
          print(f"DEBUG 1.2.4, Test mode:")
          print(f"\tDocuments: {documents}")
          print("-_-_-_-_-_-_-_-_-_-_-_-_-")
'''

In [44]:
# Shuffle the valid_sentEx df
#valid_sentEx = valid_sentEx.sample(frac=1).reset_index(drop=True)

In [73]:
#claim, documents_text, label = generate_claim_tab_data(valid_sentEx)  # Generate a claim and its associated documents for tabular data
verbose = 1
claim, label, evidence_items, documents_text, keywords, entities = get_test_claim(valid_sentEx, mode='test', verbose=1, debug=True)

# Convert documents to list of strings if it's a single string
if isinstance(documents_text, str):
    documents = documents_text.split('\n')
elif isinstance(documents_text, list):
  documents = documents_text
else:
  raise Exception(f"documents must be a list of strings, not {type(documents_text)}")

sim_thresh = 0.1
extracted_evidence, status = module_2_semi_supervised_distillation(claim, documents, entities, keywords, sim_thresh, verbose=1, debug=True)

print(f"Extracted evidence: {extracted_evidence}")
print(f"Status: {status}")

# Quick near-match function
def near_match(a, b, threshold=0.8, verbose=0):
  """
  Checks if two strings are similar based on a threshold.

  Args:
      a (str): The first string.
      b (str): The second string.
      threshold (float): The similarity threshold.

  Returns:
      bool: True if the strings are similar, False otherwise.
  """
  sim = len(set(a.split()).intersection(set(b.split()))) / max(len(a.split()), len(b.split())) >= threshold
  if verbose == 1:
      print(f"Comparing '{a}' with '{b}'\n\tat {threshold}Similarity = {sim}")
  return sim

# Match the evidence references with the extracted evidence
gold_evidence = []
for item in evidence_items:
    gold_evidence.append([None, None, item[1], item[2]])
predicted_evidence = []
extracted_evidence = list(set(extracted_evidence))
if status == "OK":
  # Check if the extracted evidence is not empty
  if extracted_evidence:
    # Iterate through the extracted evidence and evidence items
    for sentence in extracted_evidence:
        for item in evidence_items:
            if near_match(sentence, item[0], threshold=0.8, verbose=1):
                predicted_evidence.append([item[1], item[2]])
                if verbose == 1:
                    print(f"Evidence item matches extracted evidence.")
                break
            else:
                #if verbose == 1:
                    #print(f"Evidence item '{item[0]}' does not match extracted evidence '{sentence}'")
                continue
  else:
    print(f"No extracted evidence found, despite status OK.")
else:
  print(f"Status is NOT ENOUGH INFO, no evidence references found.")
  predicted_evidence = []

if verbose == 1:
    print(f"Gold evidence: {gold_evidence}")
    print(f"Predicted evidence: {predicted_evidence}")

DEBUG 1.1.1:
Evidence items: [["As of 2005 , the town 's population is 283 .", 'Angelsberg', 1, []], ['Angelsberg is a small town in the commune of Fischbach , in central Luxembourg .', 'Angelsberg', 0, ['Fischbach, Mersch', 'commune', 'Fischbach', 'Luxembourg', 'Communes of Luxembourg']]]
DEBUG 1.1.2:
	Evidence item: ["As of 2005 , the town 's population is 283 .", 'Angelsberg', 1, []]
	Entities: []
-_-_-_-_-_-_-_-_-_-_-_-_-_-
DEBUG 1.1.2:
	Evidence item: ['Angelsberg is a small town in the commune of Fischbach , in central Luxembourg .', 'Angelsberg', 0, ['Fischbach, Mersch', 'commune', 'Fischbach', 'Luxembourg', 'Communes of Luxembourg']]
	Entities: ['Fischbach, Mersch', 'commune', 'Fischbach', 'Luxembourg', 'Communes of Luxembourg']
-_-_-_-_-_-_-_-_-_-_-_-_-_-
-------------------------------------------------------------
In get_test_claim, running in test mode...
Claim: Angelsberg is a place.
Label: SUPPORTS
Evidence items: [["As of 2005 , the town 's population is 283 .", 'Angelsb

In [78]:
# Build the object for FEVER scoring

# If label is not NOT ENOUGH INFO, coin flip whether label is SUPPORTS or REFUTES for clear testing of module 2 (This is module 3 for now)
if status != "NOT ENOUGH INFO":
    if np.random.rand() < 0.5:
        predicted_label = "SUPPORTS"
    else:
        predicted_label = "REFUTES"
else:
    predicted_label = "NOT ENOUGH INFO"
pred_evdc = [['Angelsberg', 0], ['Angelsberg', 1]]

instances = [
    {
        "label": label,
        "predicted_label": predicted_label,
        "predicted_evidence": predicted_evidence,
        "evidence": [
            gold_evidence
        ],
    },
]

strict_score, label_accuracy, precision, recall, f1 = fever_score(instances)
fever_results = {
    'strict_score': strict_score,
    'label_accuracy': label_accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1
}

print("Predicted label: " + predicted_label)
for key, value in fever_results.items():
    print(f"{key}\t{value}")

Predicted label: REFUTES
strict_score	0.0
label_accuracy	0.0
precision	1.0
recall	0.0
f1	0.0


In [ ]:


# Convert extracted evidence to FEVER format
predicted_evidence = []
for sentence in extracted_evidence:
    for doc in documents:
        if sentence in doc:  # or a more robust check if sentences are preprocessed
            line_num = doc.split('\n').index(sentence)  # Assumes sentences are split by newlines in doc
            predicted_evidence.append([f"page_{documents.index(doc)}", line_num])
            break  # Stop searching once sentence is found in a document

# Construct FEVER prediction instance
prediction_instance = {
    "predicted_label": "SUPPORTS" if status == "OK" else "NOT ENOUGH INFO",  # Or REFUTES based on your logic
    "predicted_evidence": predicted_evidence,

}


# Get evidence for the claim (adapt this based on your gold standard data format)
actual_evidence = []
gold_sentences = valid_sentEx[valid_sentEx["claim"] == claim]["evidence_sentences"].iloc[0].split('\n')
for sentence in gold_sentences:
    for doc in documents:
        if sentence in doc:
            line_num = doc.split('\n').index(sentence)
            actual_evidence.append([f"page_{documents.index(doc)}", line_num])
            break

# Construct the actual evidence (gold standard) dictionary:

actual_instance = {"label": label, "evidence": [actual_evidence]}

# Calculate FEVER score
strict_score, label_accuracy, precision, recall, f1 = fever_score([prediction_instance], [actual_instance])
fever_results = {
    'strict_score': strict_score,
    'label_accuracy': label_accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1
}

print("FEVER Scores:", fever_results)
print("-------------------------------------------------------------")


if status == "OK":
    print("Extracted Evidence Sentences:")
    for sentence in extracted_evidence:
        print('\t' + sentence)
else:
    print(f"Claim classification: {status}")



In [ ]:



extracted_evidence, status = module_2_semi_supervised_distillation(claim, documents, entities, keywords, verbose=1, debug=True)

if status == "OK":
    print("Extracted Evidence Sentences:")
    for sentence in extracted_evidence:
        print('\t' + sentence)
    print(f"Claim classification: {status}")
    print(f"Label: {label}")
else:
    print(f"Claim classification: {status}")
    print(f"Label: {label}")

# Evaluation
# Assuming you have a gold standard set of sentences for evaluation

print("Evaluation Scores:")
scores, avg_score = scorer(documents, set(extracted_evidence))
for sentence, score in scores.items():
    print(f"Sentence: {sentence}")
    print(f"Scores: {score}")
    print("-------------------------------------------------------------")
print(f"Average Score: {avg_score}")  # Adjust based on your scoring logic